In [ ]:
import os 
from PIL import Image 
import numpy as np
import matplotlib.pyplot as plt
import glob
import shutil
import cv2
import skimage.filters

In [ ]:
SourceFolder = r'insert file directory'
Subfolders = [f.path for f in os.scandir(SourceFolder) if f.is_dir()]
OutFolder = r'insert file directory'

In [ ]:
#Extract FITC from subfolder, copy, and save as new name in the main folder 
count = 1
for sub in Subfolders:
    for f in os.listdir(sub):
        if f == "s_C001.tif":
            #copy to main folder
            src_file = sub + r'/s_C001.tif'
            des_dir = SourceFolder 
            shutil.copy(src_file, des_dir)
            
            #rename
            des_file = SourceFolder + r'/s_C001.tif'
            new_des_file_name = SourceFolder + r"\Network" + str(count) + ".tif"
            os.rename(des_file, new_des_file_name)
            count += 1    

In [ ]:
adjpic = []
# Automate brightness n contrast
def automatic_brightness_and_contrast(image, clip_hist_percent=1):
    # Calculate grayscale histogram
    hist = cv2.calcHist([image],[0],None,[256],[0,256])
    hist_size = len(hist)
    
    # Calculate cumulative distribution from the histogram
    accumulator = []
    accumulator.append(float(hist[0]))
    for index in range(1, hist_size):
        accumulator.append(accumulator[index -1] + float(hist[index]))
    
    # Locate points to clip
    maximum = accumulator[-1]
    clip_hist_percent *= (maximum/100.0)
    clip_hist_percent /= 2.0
    
    # Locate left cut **can adjust number
    minimum_gray = 20
    while accumulator[minimum_gray] < clip_hist_percent:
        minimum_gray += 1
    
    # Locate right cut **can adjust number
    maximum_gray = hist_size -170
    while accumulator[maximum_gray] >= (maximum - clip_hist_percent):
        maximum_gray -= 1
    
    # Calculate alpha and beta values
    alpha = 255 / (maximum_gray - minimum_gray)
    beta = -minimum_gray * alpha

    auto_result = cv2.convertScaleAbs(image, alpha=alpha, beta=beta)
    return auto_result

# Manual brightness n contrast
def manual_brightness_and_contrast(image):
    alpha = 1.95 #contrast; adjust range = 1.0 -3.0
    beta = 0     #brightness; adjust range = 0-100
    manual_result = cv2.convertScaleAbs(image, alpha=alpha, beta=beta)
    return manual_result

# Low pass filter (Denoising)
def lowPassFiltering(img):
    kernel = np.ones((7,7), np.float32)/49
    gblu = cv2.GaussianBlur(img, (7,7), 0)
    return gblu

# Change to binary image and histogram adjust
def Binarychange(image):
    # automatic thresholding
    autothres = skimage.filters.threshold_otsu(image)
    
    # x value of max y 
    fig = plt.figure()
    ax = fig.add_subplot(111)
    n,b,p=ax.hist(image.ravel(), bins=256)

    nmax = np.max(n)
    arg_max = None
    for j, _n in enumerate(n):
        if _n == nmax:
            arg_max = j
            break
    x = b[arg_max]
    
    # Calculate lower threshold
    Lowerthres = (x + autothres)/2

    # Change to integer
    LT = Lowerthres.astype(int)
    
    # Convert to binary 
    bw_img = cv2.threshold(image, LT, 255, cv2.THRESH_BINARY)[1]    
    return bw_img

In [ ]:
#processing image files
for file_name in glob.glob(SourceFolder + r'\*.tif'):
    img16 = Image.open(file_name)
    img_array16 = np.asarray(img16)
    
    # Change to 8bit
    img8 = (img_array16/img_array16.max())*255
    img8 = np.uint8(img8)
    
    # Automate brightness&contrast
    autopicarray = automatic_brightness_and_contrast(img8)
    
    # Filter
    denoiseimg = lowPassFiltering(autopicarray)
    
    # Change to binary
    bwimg = Binarychange(denoiseimg)
    
    eq_img = Image.fromarray(bwimg)
    #eq_img.show()
    adjpic.append(eq_img)

In [ ]:
#Save processed image files
for (i, new) in enumerate(adjpic):
    new.save('{}{}{}'.format(OutFolder, i+1,'.tif'))